In [39]:
import pdfplumber
import pandas as pd
from collections import defaultdict

def extract_menu_table(pdf_path):
    """
    Extrai a tabela de cardápio do PDF e formata conforme o layout especificado.
    Retorna um DataFrame organizado por dia da semana.
    """
    # Mapeamento dos cabeçalhos esperados
    expected_headers = [
        "Dia",
        "Saladas",
        "Acompanhamento",
        "Feijão",
        "Guarnição",
        "Vegetariano",
        "Prato Principal"
    ]
    
    all_data = []
    
    with pdfplumber.open(pdf_path) as pdf:
        for page in pdf.pages:
            # Extrai tabelas da página
            tables = page.extract_tables() # Lembrete: Verificar o output
            print(tables)
            
            for table in tables:
                if not table:
                    continue
                
                # Encontra a linha de cabeçalho
                header_row = None
                for row in table:
                    if any("Dia" in str(cell) for cell in row):
                        header_row = [str(cell).strip() for cell in row]
                        break
                
                if not header_row:
                    continue
                
                # Processa as linhas de dados
                for row in table:
                    # Verifica se é uma linha de dados (contém um dia da semana)
                    if any(day in str(row[0]).lower() for day in ["segunda", "terça", "quarta", "quinta", "sexta"]):
                        # Limpa os dados da linha
                        cleaned_row = [
                            str(cell).strip() if cell else ""
                            for cell in row[:7]  # Considera apenas as 7 colunas esperadas
                        ]
                        
                        # Garante que temos exatamente 7 colunas
                        if len(cleaned_row) < 7:
                            cleaned_row += [""] * (7 - len(cleaned_row))
                        
                        all_data.append(cleaned_row)
    
    # Cria o DataFrame
    df = pd.DataFrame(all_data, columns=expected_headers)
    
    # Remove linhas completamente vazias
    df = df.dropna(how='all')
    
    return df

# Uso:
pdf_path = "cardapio_junho.pdf"
menu_df = extract_menu_table(pdf_path)

# Exporta para CSV
output_csv = "cardapio_formatado.csv"
menu_df.to_csv(output_csv, index=False, encoding='utf-8-sig')

print(f"Cardápio extraído e salvo em '{output_csv}'.")
print("\nExemplo dos dados extraídos:")
print(menu_df.head())

df


[[['', 'segunda-feira, 2 de junho de', '', '', 'terça-feira, 3 de junho de', '', '', 'quarta-feira, 4 de junho', '', '', 'quinta-feira, 5 de', '', '', 'sexta-feira, 6 de junho', ''], [None, '2025', None, None, '2025', None, None, 'de 2025', None, None, 'junho de 2025', None, None, 'de 2025', None]], [['Saladas', 'Acelga com tomate picado', 'Chicória com cenoura\nralada', 'Alface com rabanete', 'Almeirão com cenoura', '', 'Alface com repolho', ''], [None, None, None, None, None, None, '', None], [None, 'Beterraba cozida', 'Chuchu colorido\nFruta', 'Grão de bico com cenoura\ncozida', 'Repolho cozido\nFruta', 'Abobrinha colorida', None, None], [None, None, None, None, None, None, 'Abobrinha colorida', None], [None, None, None, None, None, None, '', None], [None, None, None, None, None, '', '', ''], ['Acompanhamento', 'Arroz Branco', 'Arroz Branco', 'Arroz Branco/arroz\ncolorido', 'Arroz Branco', 'Arroz Branco', None, None], [None, 'Feijão Preto', 'Feijão Carioca', 'Feijão Preto', 'Feijão 

,Saladas,Acelga com tomate picado,Chicória com cenoura\nralada,Alface com rabanete,Almeirão com cenoura,,Alface com repolho,
0,None,None,None,None,None,None,,None
1,None,Beterraba cozida,Chuchu colorido\nFruta,Grão de bico com cenoura\ncozida,Repolho cozido\nFruta,Abobrinha colorida,None,None
2,None,None,None,None,None,None,Abobrinha colorida,None
3,None,None,None,None,None,None,,None
4,None,None,None,None,None,,,
5,Acompanhamento,Arroz Branco,Arroz Branco,Arroz Branco/arroz\ncolorido,Arroz Branco,Arroz Branco,None,None
6,None,Feijão Preto,Feijão Carioca,Feijão Preto,Feijão Preto,Feijão Preto,None,None
7,None,None,None,None,None,,,
8,Guarnição,Aipim souté,Espaguete ao molho sugo,Quirerinha temperada,Batata doce\ncaramelada,Macarrão com ervas\nfinas,None,None
9,None,None,None,None,None,,,


In [40]:
# ℹ️ Informações gerais sobre colunas, tipos e nulos
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13 entries, 0 to 12
Data columns (total 8 columns):
 #   Column                       Non-Null Count  Dtype 
---  ------                       --------------  ----- 
 0   Saladas                      4 non-null      object
 1   Acelga com tomate picado     6 non-null      object
 2   Chicória com cenoura
ralada  6 non-null      object
 3   Alface com rabanete          6 non-null      object
 4   Almeirão com cenoura         6 non-null      object
 5                                10 non-null     object
 6   Alface com repolho           7 non-null      object
 7                                4 non-null      object
dtypes: object(8)
memory usage: 964.0+ bytes
